# Analyze Sentiment in Product Reviews

In [1]:
#dataset amazonProducts: https://data.world/datafiniti/amazon-and-best-buy-electronics

In [2]:
## Instalar bibliotecas necessarias

!pip install NRCLex
!pip install -U spacy
!pip install NRCLex
!python3 -m spacy download en_core_web_md

'pip' is not recognized as an internal or external command,
operable program or batch file.
'pip' is not recognized as an internal or external command,
operable program or batch file.
'pip' is not recognized as an internal or external command,
operable program or batch file.
Python n�o encontrado; executar sem argumentos para instalar a partir da Microsoft Store ou desativar este atalho em Defini��es > Gerir Aliases de Execu��o da Aplica��o.


In [3]:
## Imports necessarios

import pandas as pd 
from nrclex import NRCLex
import nltk
nltk.download('punkt')

import spacy

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [36]:
## Carregar dataset
data = pd.read_csv('100_reviews_to_test.csv')


In [37]:
data.head()

,Unnamed: 0,review_text,classification_NRC
0,100,\n Very nice product with promt delivary the ...,positive
1,101,\n Ordered on 26th Sep afternoon and Amazon d...,positive
2,102,\n Fingerprint Reader is very fast when compa...,positive
3,103,\n BEST FIRST 5G PHONE UNDER 30000 Rs/- MADE ...,fear
4,104,\n Overall perfomance of the phone is amazing...,positive


In [38]:
data.drop('Unnamed: 0', inplace=True, axis=1) 






In [39]:

data.head()

,review_text,classification_NRC
0,\n Very nice product with promt delivary the ...,positive
1,\n Ordered on 26th Sep afternoon and Amazon d...,positive
2,\n Fingerprint Reader is very fast when compa...,positive
3,\n BEST FIRST 5G PHONE UNDER 30000 Rs/- MADE ...,fear
4,\n Overall perfomance of the phone is amazing...,positive


### Testar o NRCLex

In [40]:
review_1 = NRCLex('the product arrive to late')
review_2 = NRCLex('The product quality is terrible.')

r1 = review_1.top_emotions
r2 = review_2.top_emotions

print(r2)


[('fear', 0.2), ('anger', 0.2), ('negative', 0.2), ('sadness', 0.2), ('disgust', 0.2)]


# Análise de sentimentos as reviews do dataset

* Import das bibliotecas e do corpora necessário
* Definição da **função de tokanização**
* Definição da **função de lematização**
  - Utiliza a biblioteca ``spacy``
  - normaliza as palavras em minúsculo e retira as **stopwords**
* Definição da **função de avaliação** texto usando o corpus do NRC com todas as emoções definidas.

In [41]:
nlp = spacy.load('en_core_web_md')

In [42]:
# Função que faz lematização, tokenização, retira pontuação e stop words, coloca o texto em minusculas

def dataPrep(review):
    doc = nlp(review)  
  #juntar os tokens obtidos na lematização numa só frase 
  #(token.pos_ == 'ADJ' or token.pos_ == 'VERB')
    res = " ".join([token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct])   
    return res

In [43]:
## Função que utiliza o NRClex para dar uma lista de sentimentos associados a uma review

def sentimentAnalysis(review):
    
  review = NRCLex(review)

    
  return dict(sorted(review.affect_frequencies.items(), key=lambda item: item[1], reverse=True))  ## ordenar resultado

 
   

In [44]:
####################################
##  Teste das função desenvolvida ##
####################################

review = 'I wasnt impressed with this smartphone. Was really excited with it and now I feel disappointed '

lem = dataPrep(review)
print(lem)

sentimentAnalysis(lem)

not impressed smartphone excited feel disappointed


{'anger': 0.1111111111111111,
 'trust': 0.1111111111111111,
 'surprise': 0.1111111111111111,
 'positive': 0.1111111111111111,
 'negative': 0.1111111111111111,
 'sadness': 0.1111111111111111,
 'disgust': 0.1111111111111111,
 'joy': 0.1111111111111111,
 'anticipation': 0.1111111111111111,
 'fear': 0.0,
 'anticip': 0.0}

## Adicionar ao dataset uma nova coluna com a análise de sentimentos

In [45]:

## Criar colunas para cada emoção calculada pelo NRC

data['classification_NRC'] = " "


In [46]:
len (data[100:200])

0

In [48]:
## ATUALIZAR O DATASET COM AS EMOÇOES RECEBIDAS PELO NRC 
## Para cada review identifica-se o conjunto de emoções associadas

def datasetUpdate (data):
    i = 0
    for (ind, row)  in data.iterrows(): 
        review = row['review_text']

        #print(review)
        
        res = dataPrep(review)

        emotions = sentimentAnalysis(res)
        
        #print(emotions)

        for x in emotions:
            data.loc[data['review_text'] == review, 'classification_NRC'] = x
            break
       
    return data


        



        

In [49]:
pd.options.mode.chained_assignment = None  # default='warn'


In [50]:
## retirar colunas com valores nulos

data['review_text'] = data['review_text'].fillna(" ")

In [51]:
data.drop(data.index[100:200], inplace=True)

In [52]:
data

,review_text,classification_NRC
0,\n Very nice product with promt delivary the ...,
1,\n Ordered on 26th Sep afternoon and Amazon d...,
2,\n Fingerprint Reader is very fast when compa...,
3,\n BEST FIRST 5G PHONE UNDER 30000 Rs/- MADE ...,
4,\n Overall perfomance of the phone is amazing...,
...,...,...
95,\n Just got it today. No display issues or he...,
96,\n I am using my old mobile for long time of ...,
97,\n Overall phone is good,
98,"\n Excellent spec, beautiful design, no major...",


In [53]:
data = datasetUpdate (data)

In [54]:
data


,review_text,classification_NRC
0,\n Very nice product with promt delivary the ...,positive
1,\n Ordered on 26th Sep afternoon and Amazon d...,positive
2,\n Fingerprint Reader is very fast when compa...,positive
3,\n BEST FIRST 5G PHONE UNDER 30000 Rs/- MADE ...,fear
4,\n Overall perfomance of the phone is amazing...,positive
...,...,...
95,\n Just got it today. No display issues or he...,anger
96,\n I am using my old mobile for long time of ...,anticipation
97,\n Overall phone is good,trust
98,"\n Excellent spec, beautiful design, no major...",positive


In [55]:
## Guardar o dataset com a classificação das emoçoes num csv.

data.to_csv('100_reviews.csv')